In [28]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [29]:
def stratified_K_fold(X,y,model,n_splits):
    
    
    skf = StratifiedKFold(n_splits)

    fold_no=1
    
    score={}
    
    for train_index, test_index in skf.split(X, y):
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train,y_train)
        predictions = model.predict(X_test)
        
        score[fold_no]={'accuracy' : accuracy_score(y_test,predictions),'precision' : precision_score(y_test,predictions),
                        'recall' : recall_score(y_test,predictions), 'f1' : f1_score(y_test,predictions),
                        'specificity' : recall_score(y_test,predictions,pos_label=0)
                           }
        fold_no += 1
    score = pd.DataFrame.from_dict(score, orient = 'index')
    score.index.name = 'flod'
    return score
    

In [44]:
from sklearn.model_selection import ParameterGrid
def param_optimization(Model,X,y,n_splits,**param): 
    if False in [isinstance(key,str) for key in param.keys()]:
        raise ValueError("Keys must be string ")
    if False in [isinstance(value,list) for value in param.values()] and  False in [isinstance(value,tuple) for value in param.values()]:
        raise ValueError("Values must be list or tuples ")
        
    else:
        
        param_grid=ParameterGrid(param)
        df=pd.DataFrame(param_grid)
        for params in param_grid:
            model=SVC(**params)
            scores=stratified_K_fold(X,y,model,n_splits)
            for s in scores:
                df['mean '+ s]= scores[s].mean()
                df['std'+ s]= scores[s].std()
                df['min' + s] = scores[s].min()
                df['max' + s] = scores[s].max()
                df['median'+s]=scores[s].median()
        return df

In [45]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1])
n_splits=2 
tuned_parameters ={"kernel": ['rbf','linear'], "gamma": [1e-3, 1e-4], "C": [1, 10, 100, 1000], "C": [1, 10, 100, 1000]}


    
param_optimization(SVC,X,y,n_splits,**tuned_parameters)
    


,C,gamma,kernel,mean accuracy,stdaccuracy,minaccuracy,maxaccuracy,medianaccuracy,mean precision,stdprecision,...,mean f1,stdf1,minf1,maxf1,medianf1,mean specificity,stdspecificity,minspecificity,maxspecificity,medianspecificity
0,1,0.0010,rbf,0.5,0.0,0.5,0.5,0.5,0.5,0.0,...,0.666667,0.0,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0
1,1,0.0010,linear,0.5,0.0,0.5,0.5,0.5,0.5,0.0,...,0.666667,0.0,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0
2,1,0.0001,rbf,0.5,0.0,0.5,0.5,0.5,0.5,0.0,...,0.666667,0.0,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0
3,1,0.0001,linear,0.5,0.0,0.5,0.5,0.5,0.5,0.0,...,0.666667,0.0,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0
4,10,0.0010,rbf,0.5,0.0,0.5,0.5,0.5,0.5,0.0,...,0.666667,0.0,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0
5,10,0.0010,linear,0.5,0.0,0.5,0.5,0.5,0.5,0.0,...,0.666667,0.0,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0
6,10,0.0001,rbf,0.5,0.0,0.5,0.5,0.5,0.5,0.0,...,0.666667,0.0,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0
7,10,0.0001,linear,0.5,0.0,0.5,0.5,0.5,0.5,0.0,...,0.666667,0.0,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0
8,100,0.0010,rbf,0.5,0.0,0.5,0.5,0.5,0.5,0.0,...,0.666667,0.0,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0
9,100,0.0010,linear,0.5,0.0,0.5,0.5,0.5,0.5,0.0,...,0.666667,0.0,0.666667,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0
